## Data Inspection
- Goal: understand quantity and sparsity of data across patients

In [1]:
# Set Global Consts
import os
import pandas as pd
import numpy as np
from etl import load_csvs_to_dataframe_dict

In [17]:
fn_to_df_map = load_csvs_to_dataframe_dict()

# Get # of unique instances per concept_id
## get "good enough" largest df size
person_count = len(fn_to_df_map['person.csv']['person_id'])
clin_concept_count = 0
## for each csv with clinical concepts, get unique counts
for fn in FILENAME_CLIN_CONCEPT_MAP:
    df = fn_to_df_map[fn]
    for col in FILENAME_CLIN_CONCEPT_MAP[fn]:
        clin_concept_count += len(df[col].unique())

## assume worst case that each person has once occurrence of every possible clinical concept
print(person_count, clin_concept_count) # 1251 person_ids * 3161 unique concepts = 3,954,411 max num of rows

## init empty dataframe df_all
idx = range(person_count * clin_concept_count)
cols = ['person_id', 'concept_id']
df_all = pd.DataFrame(index=idx, columns=cols)
## populate df_all (unique person_id per concept_id)
count = 0
for fn in FILENAME_CLIN_CONCEPT_MAP:
    df = fn_to_df_map[fn]
    for col in FILENAME_CLIN_CONCEPT_MAP[fn]:
        # pre-process: get all non-unique (person_id, concept_id) pairs
        df_sliced = df.loc[:, ['person_id', col]]
        df_sliced = df_sliced.dropna()
        df_sliced = df_sliced.rename(columns={col: 'concept_id'})
        ## set appropriate index
        n_rows = len(df_sliced)
        idx = pd.Series(range(count, count+n_rows))
        df_sliced = df_sliced.set_index(idx) 

        # append to df_all
        df_all.iloc[idx, :] = df_sliced
        count += n_rows

## remove NaN
df_all = df_all.dropna().astype('int')
print(df_all)

1251 3161
         person_id  concept_id
0              909      380378
1             1196       75909
2              156      438409
3             1064      435875
4              925       80502
...            ...         ...
1590019          2       32209
1590020        424       32209
1590021        168       32209
1590022       1198       32209
1590023        489       32209

[1590024 rows x 2 columns]


In [11]:
# Get count of unique person_id per concept_id
df_all_summary = df_all.drop_duplicates(keep='first')\
    .groupby(['concept_id'])\
    .agg({'person_id': 'count'})\
    .rename(columns={'person_id': 'unique_pid_count'})\
    .sort_values('unique_pid_count', ascending=False)

## add concept_name, table labels from data_dict
### names
concept_ids = df_all_summary.reset_index().loc[:, 'concept_id']
cid_to_name = lambda cid: CONCEPT_ID_TO_NAME_MAP[cid] if cid in CONCEPT_ID_TO_NAME_MAP else pd.NA
concept_names = concept_ids.apply(cid_to_name).rename('concept_name')
concept_ids_with_names = pd.concat([concept_ids, concept_names], axis=1).set_index('concept_id')
df_all_summary.insert(0, "concept_name", concept_ids_with_names)

### table
cid_to_table = lambda cid: CONCEPT_ID_TO_TABLE_MAP[cid] if cid in CONCEPT_ID_TO_TABLE_MAP else pd.NA
concept_table = concept_ids.apply(cid_to_table).rename('from_table')
concept_ids_with_table = pd.concat([concept_ids, concept_table], axis=1).set_index('concept_id')
df_all_summary.insert(1, "from_table", concept_ids_with_table)


# Get count of average # of occurrences per person with the given concept_id
m = len(df_all)
df_ones = pd.DataFrame(np.ones((m, 1)))
df_all_w_ones = pd.concat([df_all, df_ones], axis=1)
df_all_avg = df_all_w_ones.groupby(['concept_id', 'person_id'])\
    .agg(['sum'])\
    .reset_index()\
    .rename(columns={0: 'avg_per_pid'})\
    .groupby(['concept_id'])\
    .agg(['mean'])

## clean-up formatting a bit
df_all_avg.columns.droplevel([1,2]) # remove multiindex
df_all_avg = df_all_avg.loc[:, 'avg_per_pid'] # keep only avg_per_pid

## add to df_all_summary
df_all_summary.insert(1, "avg_per_pid", df_all_avg)


# Save to csv
df_all_summary.to_csv(DATA_PATH + '/concept_summary.csv')

print(df_all_summary)

                                                 concept_name  ...  unique_pid_count
concept_id                                                     ...                  
44814724                                                  NaN  ...              1251
4033240                                                   NaN  ...              1251
44818518                                                  NaN  ...              1251
32019                                                     NaN  ...              1251
44818702                                                  NaN  ...              1251
...                                                       ...  ...               ...
2788037      Respiratory Ventilation, 24-96 Consecutive Hours  ...                 1
2784252     Detachment at Right Lower Leg, High, Open Appr...  ...                 1
2784255     Detachment at Left Lower Leg, High, Open Approach  ...                 1
2784265     Detachment at Right Foot, Partial 2nd Ray, Ope...  ..

## Clustering Analysis - Finding Highest-Separation Feature Combinations
- Goal: find set of features that result in best PCA Clustering
- Pipeline: pick set of features (profile) -> run PCA -> manually view data -> run K-Means/GMM to cluster -> score clusters: use this as predictive measurement